In [ ]:
# 1️⃣ Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2️⃣ Start Ollama in background
!nohup ollama serve > /dev/null 2>&1 &


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama serve


Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [ ]:
!ollama pull deepseek-r1:14b


In [ ]:
!ollama run deepseek-r1:14b "Hello from DeepSeek!"


Hello! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. For comprehensive details about our models and products, we invite you to consult our official documentation.



In [ ]:
# ============================================
# 🚀 1️⃣ Install Ollama (Skip if already done)
# ============================================
!curl -fsSL https://ollama.com/install.sh | sh

# ============================================
# 🚀 2️⃣ Start Ollama Server in Background
# ============================================
!nohup ollama serve > /dev/null 2>&1 &

# ============================================
# 🚀 3️⃣ (Optional) Pull DeepSeek Model (Skip if already downloaded)
# ============================================
!ollama pull deepseek-r1:14b

# ============================================
# 🚀 4️⃣ Test Model (Optional)
# ============================================
!ollama run deepseek-r1:14b "Hello from DeepSeek!"

# ============================================
# 🌐 5️⃣ Install Cloudflare Tunnel (cloudflared)
# ============================================
!curl -fsSL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

# ============================================
# 🌍 6️⃣ Expose Ollama to Internet via Cloudflare
# ============================================
import subprocess, re

# Run Cloudflare tunnel and capture its output
process = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:11434"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("⏳ Starting Cloudflare Tunnel... Please wait 10–20 seconds.\n")

# Capture the public URL
public_url = None
for line in iter(process.stdout.readline, ""):
    print(line.strip())
    if "trycloudflare.com" in line:
        public_url = re.search(r"https://[0-9a-zA-Z\-]+\.trycloudflare\.com", line)
        if public_url:
            public_url = public_url.group(0)
            print(f"\n✅ Public URL (Use this in your Django backend): {public_url}\n")
            break

# ============================================
# ✅ 7️⃣ Example Use (inside Django)
# ============================================
# Once the above URL appears, use it in your backend like this:
#
# from langchain_ollama.chat_models import ChatOllama
# model = ChatOllama(
#     base_url="https://your-tunnel-name.trycloudflare.com",
#     model="deepseek-r1:14b",
#     temperature=0.3
# )
# response = model.invoke("Hello DeepSeek!")
# print(response)
#
# ============================================
# ✅ Done! Keep this cell running to keep the tunnel alive.
# ============================================


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

Hello! How can I assist you today? 😊

⏳ Starting Cloudflare Tunnel... Please wait 10–20 seconds.

2025-10-14T14:07:01Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in

In [ ]:
import subprocess
import re
import sys
import threading
import shutil
import os
import time
from pathlib import Path

# ============================================
# 🧩 Utility function: Check if command exists
# ============================================
def is_installed(cmd):
    return shutil.which(cmd) is not None


# ============================================
# 🧱 0️⃣ Ensure Ollama config allows Cloudflare
# ============================================
def ensure_ollama_config():
    config_path = Path.home() / ".ollama" / "config.yaml"
    config_path.parent.mkdir(parents=True, exist_ok=True)

    config_content = """OLLAMA_HOST: 0.0.0.0
OLLAMA_ORIGINS:
  - http://localhost
  - https://localhost
  - http://127.0.0.1
  - https://127.0.0.1
  - https://*.trycloudflare.com
"""

    with open(config_path, "w", encoding="utf-8") as f:
        f.write(config_content)
    print(f"✅ Ollama config written to: {config_path}\n")


# ============================================
# 🚀 1️⃣ Install Ollama (if not installed)
# ============================================
def install_ollama():
    if not is_installed("ollama"):
        print("⏳ Installing Ollama... Logs will appear below.\n")
        install_proc = subprocess.Popen(
            ["bash", "-c", "curl -fsSL https://ollama.com/install.sh | sh"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        for line in iter(install_proc.stdout.readline, ""):
            if line:
                print(line.strip())
        install_proc.wait()
    else:
        print("✅ Ollama already installed.\n")


# ============================================
# 🚀 2️⃣ Pull Model (if missing)
# ============================================
def pull_model(model_name):
    try:
        print(f"🔍 Checking if model '{model_name}' exists...\n")
        result = subprocess.run(
            ["ollama", "list"], capture_output=True, text=True
        )
        if model_name not in result.stdout:
            print(f"⏳ Pulling model {model_name}... Logs will appear below.\n")
            pull_proc = subprocess.Popen(
                ["ollama", "pull", model_name],
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True
            )
            for line in iter(pull_proc.stdout.readline, ""):
                if line:
                    print(line.strip())
            pull_proc.wait()
        else:
            print(f"✅ Model '{model_name}' already available.\n")
    except Exception as e:
        print(f"⚠️ Error checking/pulling model: {e}")


# ============================================
# 🚀 3️⃣ Start Ollama Server (with open origins)
# ============================================
def start_ollama_server():
    print("⏳ Starting Ollama server... Logs will appear below.\n")

    ollama_proc = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        env={**os.environ, "OLLAMA_ORIGINS": "*", "OLLAMA_HOST": "0.0.0.0"}
    )

    def print_ollama_logs():
        for line in iter(ollama_proc.stdout.readline, ""):
            if line:
                print(f"[Ollama] {line.strip()}")

    threading.Thread(target=print_ollama_logs, daemon=True).start()
    return ollama_proc


# ============================================
# 🌐 4️⃣ Install cloudflared (if missing)
# ============================================
def install_cloudflared():
    if not shutil.which("cloudflared"):
        print("\n⏳ Installing Cloudflare Tunnel (cloudflared)...\n")
        subprocess.run([
            "curl", "-fsSL",
            "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
            "-o", "cloudflared"
        ])
        subprocess.run(["chmod", "+x", "cloudflared"])
    else:
        print("✅ cloudflared already installed.\n")


# ============================================
# 🌍 5️⃣ Start Cloudflare Tunnel
# ============================================
def start_cloudflare_tunnel():
    print("⏳ Starting Cloudflare tunnel... Logs will appear below.\n")
    cf_proc = subprocess.Popen(
        ["./cloudflared", "tunnel", "--url", "http://localhost:11434"],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    public_url = None
    for line in iter(cf_proc.stdout.readline, ""):
        if line:
            print(f"[Cloudflare] {line.strip()}")
            if "trycloudflare.com" in line and not public_url:
                match = re.search(r"https://[0-9a-zA-Z\-]+\.trycloudflare\.com", line)
                if match:
                    public_url = match.group(0)
                    print(f"\n✅ Public URL ready: {public_url}\n")
    return public_url


# ============================================
# ✅ MAIN EXECUTION FLOW
# ============================================
if __name__ == "__main__":
    ensure_ollama_config()
    install_ollama()

    model_name = "deepseek-r1:14b"
    pull_model(model_name)

    ollama_proc = start_ollama_server()
    time.sleep(8)  # give server some time to boot

    # Re-check model once server is up (safety)
    pull_model(model_name)

    install_cloudflared()
    public_url = start_cloudflare_tunnel()

    print("\n✅ Ollama server and Cloudflare tunnel are running.\n")
    print(f"💡 Use this public URL in Django or API requests:\n👉 {public_url}\n")


✅ Ollama config written to: /root/.ollama/config.yaml

✅ Ollama already installed.

🔍 Checking if model 'deepseek-r1:14b' exists...

⏳ Pulling model deepseek-r1:14b... Logs will appear below.

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
⏳ Starting Ollama server... Logs will appear below.

[Ollama] time=2025-10-15T17:13:10.743Z level=INFO source=routes.go:1481 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OL

✅ Ollama already installed.

🔍 Checking if Ollama server is running...
✅ Ollama server already running.
✅ Model 'deepseek-r1:14b' already available.

🧪 Testing model response (short prompt)...
✅ Model responded:
 Hello! How can I assist you today? 😊

 ...

✅ cloudflared already installed.

🌍 Launching Cloudflare Tunnel to expose Ollama (port 11434)...
2025-10-14T14:14:47Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-14T14:14:47Z INF Requesting new quic